In [1]:
pip install wbgapi pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
import wbgapi as wb
import pandas as pd
import altair as alt

In [ ]:
# World Bank API wbgapi

# Ver todos los indicadores (son miles)
#wb.series.info()

# Buscar indicadores por palabra clave
#indicators = wb.series.list(q='account')

# Ver detalles de un indicador específico
#wb.series.metadata.get('NY.GDP.PCAP.CD')

<generator object list at 0x1181cda20>

In [79]:
# 1. Define the indicators from Global Findex and World Bank
# fx.acc.t.d: % of adults with a bank account
# fx.sav.fin.t.d: % of adults who saved at a financial institution
# NY.GDP.PCAP.CD: GDP per capita (current US$)
indicators = {
    'FX.OWN.TOTL.ZS': 'Account_Ownership',
    'NY.GNS.ICTR.ZS': 'Gross_Savings',
    'NY.GDP.PCAP.CD': 'GDP_per_Capita'
}

In [80]:
year_to_fetch = 2024

In [ ]:
df = wb.data.DataFrame(indicators.keys(), time=year_to_fetch, labels=True, skipAggs=True).reset_index()

In [82]:
df

,economy,Country,FX.OWN.TOTL.ZS,NY.GDP.PCAP.CD,NY.GNS.ICTR.ZS
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213
1,ZMB,Zambia,72.702425,1187.109434,34.421066
2,YEM,"Yemen, Rep.",NaN,NaN,NaN
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263
4,VIR,Virgin Islands (U.S.),NaN,NaN,NaN
...,...,...,...,...,...
212,AND,Andorra,NaN,49303.649167,NaN
213,ASM,American Samoa,NaN,NaN,NaN
214,DZA,Algeria,35.290107,5752.990767,39.419856
215,ALB,Albania,46.069251,11377.775743,22.501612


In [86]:
# 3. Cleaning and renaming
df = df.rename(columns={
        'economy': 'ISO3',
        'Country': 'Country_Name',
        'NY.GNS.ICTR.ZS': 'Gross_Savings_GDP',
        'FX.OWN.TOTL.ZS': 'Account_Ownership',
        'NY.GDP.PCAP.CD': 'GDP_per_Capita'
    })

In [87]:
df

,ISO3,Country_Name,Account_Ownership,GDP_per_Capita,Gross_Savings_GDP
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213
1,ZMB,Zambia,72.702425,1187.109434,34.421066
2,YEM,"Yemen, Rep.",NaN,NaN,NaN
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263
4,VIR,Virgin Islands (U.S.),NaN,NaN,NaN
...,...,...,...,...,...
212,AND,Andorra,NaN,49303.649167,NaN
213,ASM,American Samoa,NaN,NaN,NaN
214,DZA,Algeria,35.290107,5752.990767,39.419856
215,ALB,Albania,46.069251,11377.775743,22.501612


In [89]:
# Limpieza: Eliminar nulos para asegurar que el Bubble Chart funcione
df_clean = df.dropna(subset=['Gross_Savings_GDP', 'Account_Ownership', 'GDP_per_Capita'])
df_clean

,ISO3,Country_Name,Account_Ownership,GDP_per_Capita,Gross_Savings_GDP
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213
1,ZMB,Zambia,72.702425,1187.109434,34.421066
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263
5,VNM,Viet Nam,70.550753,4717.290287,36.690108
8,UZB,Uzbekistan,59.658383,3161.700106,28.297617
...,...,...,...,...,...
206,AUS,Australia,98.010378,64603.985631,22.922852
208,ARM,Armenia,71.373473,8556.214070,19.235686
209,ARG,Argentina,81.744245,13969.783660,16.697117
214,DZA,Algeria,35.290107,5752.990767,39.419856


In [92]:
# Guardar para Vega-Lite
df_clean.to_csv('ownership_savings.csv', index=False)
    
print(f"¡Éxito! Datos guardados para {len(df_clean)} países.")
print(df_clean[['Country_Name', 'Gross_Savings_GDP', 'Account_Ownership']].head())

¡Éxito! Datos guardados para 111 países.
         Country_Name  Gross_Savings_GDP  Account_Ownership
0            Zimbabwe          10.700213          49.521602
1              Zambia          34.421066          72.702425
3  West Bank and Gaza           1.883263          39.620984
5            Viet Nam          36.690108          70.550753
8          Uzbekistan          28.297617          59.658383


In [ ]:


# 5. Data Cleaning: Remove rows with missing values
# Important for the Bubble Chart to have all three coordinates (X, Y, Size)
df_clean = df.dropna(subset=['Account_Ownership', 'Formal_Savings', 'GDP_per_Capita'])

# 6. Export to CSV for your project
# This file will be the data source for your Vega-Lite charts
df_clean.to_csv('data_inclusion_vs_usage.csv', index=False)

print(f"Success! Processed {len(df_clean)} countries.")
print(df_clean.head())